![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

# *Final Project: Contextual Bandits - Algorithm Analysis*

## Reinforcement Learning

#### Authors: **Timothy Cassel, Marvin Ernst, Oliver Tausendschön**

Date: July 2, 2025

Instructors: *Hamish Flynn and Vincent Adam*

In this notebook, we evaluate and analyze contextual bandit algorithms using the **Open Bandit Dataset (OBD)**. The main focus lies on implementing and benchmarking **Thompson Sampling (TS)** variants under realistic logged feedback conditions.

We perform the following:
- Implement **Thompson Sampling for linear bandits** using Bayesian linear regression
- Compare TS to **LinUCB** and **Random** policies
- Evaluate offline performance using:
  - **Inverse Propensity Scoring (IPS)**
  - **Doubly Robust (DR)** estimators
- Assess exploration behavior and learning stability
 
This notebook provides additional insights into the empirical behavior of Bayesian bandit strategies and whether **TS** can outperform **UCB** in sparse, realistic environments.

---

## 1. Setup and Imports

Importing the relevant libraries:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict
import random
import os

Set seeds for reproducibility:

In [2]:
np.random.seed(42)
random.seed(42)

## 2. Load Processed Data

(We have the preprocessed version in `processed_top3/`)

In [6]:
X_context = np.load("processed_top3/context_reduced.npy")
actions = np.load("processed_top3/actions.npy")
rewards = np.load("processed_top3/rewards.npy")
pscores = np.load("processed_top3/pscores.npy")
meta = np.genfromtxt("processed_top3/meta.txt", skip_header=1)

n_rounds, context_dim = X_context.shape
n_arms = len(np.unique(actions))

print(f"Context shape: {X_context.shape}, #Arms: {n_arms}")

Context shape: (53988, 12), #Arms: 3


## 3. Implement Thompson Sampling

**To be continued!**